In [1]:
pip install pytorch_metric_learning

     |████████████████████████████████| 106 kB 12.1 MB/s 


In [2]:
import sys
import torch, torchvision, math, time, argparse, os
import random
import numpy as np

from google.colab import drive
drive.mount('/gdrive')

root_folder = os.path.abspath('/gdrive/MyDrive/colab')
sys.path.append(root_folder)

import dataset, net

from net.resnet import *

Mounted at /gdrive


In [3]:

from torch import nn
import torch.nn.functional as F
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

from pytorch_metric_learning.losses.contrastive_loss import ContrastiveLoss
from pytorch_metric_learning.distances.lp_distance import LpDistance
from pytorch_metric_learning.reducers.avg_non_zero_reducer import AvgNonZeroReducer
from pytorch_metric_learning.reducers.multiple_reducers import MultipleReducers
from pytorch_metric_learning.utils import loss_and_miner_utils as lmu

from tqdm import *
import random


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
datasets = 'cub'  #change this to 'cars' to perform experiment on CARS-196

data_root = root_folder + '/data/CUB_200_2011'
saveDir = root_folder + '/checkpoint/' + datasets + '/'

if not os.path.exists(saveDir):
    os.makedirs(saveDir)

print(data_root)


/gdrive/MyDrive/colab/data/CUB_200_2011


In [6]:
models = 'resnet50'  # network
sz_batch = 100       # batch size
nb_workers = 2       # number of workers for dataloader (dont change)
sz_embedding = 512   # embedding dimension
bn_freeze = 1        # freeze batch normalization
l2_norm = 1          # l2 normalize embeddings
nb_epochs = 60       # total training epochs
gpu_id = 0           # you will use one gpu, this is the ID of the gpu
warm = 5             # number of epochs of warmup
lr_decay_step = 5    # decay learning rate after 5 epochs
lr_decay_gamma = 0.1 # learning rate decay factor

In [7]:
# load your training set
# print(data_root)
trn_dataset = dataset.load(
            name = datasets,
            root = data_root,
            mode = 'train',
            transform = dataset.utils.make_transform(
                is_train = True, 
                is_inception = (models == 'bn_inception')
            ))


dl_tr = torch.utils.data.DataLoader(
    trn_dataset,
    batch_size = sz_batch,
    shuffle = True,
    num_workers = nb_workers,
    drop_last = True,
    pin_memory = True
)
print('Random Sampling')

ev_dataset = dataset.load(
            name = datasets,
            root = data_root,
            mode = 'eval',
            transform = dataset.utils.make_transform(
                is_train = False, 
                is_inception = (models == 'bn_inception')
            ))

dl_ev = torch.utils.data.DataLoader(
    ev_dataset,
    batch_size = sz_batch,
    shuffle = False,
    num_workers = nb_workers,
    pin_memory = True
)


nb_classes = trn_dataset.nb_classes()
print(nb_classes)

Random Sampling
100


In [8]:
model = Resnet50(embedding_size=sz_embedding, pretrained=True, is_norm=l2_norm, bn_freeze = bn_freeze)

'''
load model to GPU

for initial part of your experiment, comment out Line 112 and only use it for the final run

'''
model.to(device) 

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Resnet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [9]:
def margin(x, y):
    return x - y


lr = 1e-4
weight_decay = 1e-4
margin_value = 0.5
criterion = ContrastiveLoss(neg_margin=0.5)
distance = LpDistance()
reducer_dict = {"pos_loss" : AvgNonZeroReducer(), "neg_loss" : AvgNonZeroReducer()}
reducer = MultipleReducers(reducer_dict)


param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters()))) if gpu_id != -1 else 
                 list(set(model.module.parameters()).difference(set(model.module.model.embedding.parameters())))},
    {'params': model.model.embedding.parameters() if gpu_id != -1 else model.module.model.embedding.parameters(), 'lr':float(lr) * 1},
]
# exit()
opt = torch.optim.Adam(param_groups, lr=float(lr), weight_decay = weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=lr_decay_step, gamma = lr_decay_gamma)


losses_list = []
best_recall=[0]
best_epoch = 0

In [10]:
for epoch in range(0, nb_epochs):
    model.train()
    bn_freeze = bn_freeze
    if bn_freeze:
        modules = model.model.modules() if gpu_id != -1 else model.module.model.modules()
        for m in modules: 
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    losses_per_epoch = []
    
    # Warmup: Train only new params, helps stabilize learning.
    if warm > 0:
        if gpu_id != -1:
            unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())
        else:
            unfreeze_model_param = list(model.module.model.embedding.parameters()) + list(criterion.parameters())

        if epoch == 0:
            for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
                param.requires_grad = False
        if epoch == warm:
            for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
                param.requires_grad = True


    pbar = tqdm(enumerate(dl_tr))
    

    for batch_idx, (inputs, target) in pbar: 

        inputs = inputs.cuda()
        target = target.cuda()  

        #get your pairs
        a1, p, a2, n = lmu.get_all_pairs_indices(target)

        # pass inputs into the model
        out = model(inputs)

        # compute similarity with respect to embeddings
        embedding_similarity = distance(out)
        
        
        # Get similarity of positive and negative pairs
      
        pos_pair_dist, neg_pair_dist = [], []
        if len(a1) > 0:
            pos_pair_dist = embedding_similarity[a1, p]
        if len(a2) > 0:
            neg_pair_dist = embedding_similarity[a2, n]

        indices_tuple = (a1, p, a2, n)
        pos_pairs = lmu.pos_pairs_from_tuple(indices_tuple)
        neg_pairs = lmu.neg_pairs_from_tuple(indices_tuple)

        
        # compute loss using margin value

        if len(pos_pair_dist) > 0:
            pos_loss = torch.nn.functional.relu(margin(pos_pair_dist, 0.0))
        if len(neg_pair_dist) > 0:
            neg_loss = torch.nn.functional.relu(margin(margin_value, neg_pair_dist))


        loss_dict =  {
            "pos_loss": {
                "losses": pos_loss,
                "indices": pos_pairs,
                "reduction_type": "pos_pair",
            },
            "neg_loss": {
                "losses": neg_loss,
                "indices": neg_pairs,
                "reduction_type": "neg_pair",
            },
        }


        loss = reducer(loss_dict, embedding_similarity, target)

        # set gradients to 0
        
        opt.zero_grad()

        # compute gradients
        loss.backward()

        # clip gradients for stability
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())

        # update weights
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(dl_tr),
                100. * batch_idx / len(dl_tr),
                loss.item()))
        

    mean_loss = np.mean(losses_per_epoch)
    print('------> epoch: {} -->  loss = {:.4f} '.format(epoch,mean_loss))
    scheduler.step()
    
    if(epoch >= 0):
        with torch.no_grad():
            print("**Evaluating...**")
            if datasets == 'Inshop':
                Recalls = utils.evaluate_cos_Inshop(model, dl_query, dl_gallery)
            elif datasets != 'SOP':
                Recalls = utils.evaluate_cos(model, dl_ev)
            else:
                Recalls = utils.evaluate_cos_SOP(model, dl_ev)
                
        
        # Best model save
        if best_recall[0] < Recalls[0]:
            
            print('Saving..')
            best_recall = Recalls
            best_epoch = epoch
            if not os.path.exists(saveDir):
                os.makedirs(saveDir)
            state = {
                'model': model,
                'Recall': Recalls,
                'epoch': epoch,
                'rng_state': torch.get_rng_state()
                }
       
            # torch.save(state, saveDir + 'baseline_R50_d512.t7')


print(best_recall)

Train Epoch: 0 [4/58 (5%)] Loss: 0.642291: : 4it [01:50, 27.53s/it]


KeyboardInterrupt: ignored